# SQL


In [ ]:
import qrcode
import os

def generate_qr_codes(base_url, route_id, stop_count, out_dir="qrcodes"):
    os.makedirs(out_dir, exist_ok=True)

    for stop_order in range(1, stop_count + 1):
        url = f"{base_url}/routes/{route_id}/stop/{stop_order}"
        img = qrcode.make(url)
        filename = os.path.join(out_dir, f"route{route_id}_stop{stop_order}.png")
        img.save(filename)
        print("已產生:", filename, "→", url)

# 使用範例
if __name__ == "__main__":
    # 你的 ngrok host，或正式網域
    base = "https://f1c2fbf338af.ngrok-free.app"
    generate_qr_codes(base_url=base, route_id=2, stop_count=8)


In [4]:
import requests
import json
import hashlib
from base64 import b64encode, b64decode
from Crypto.Cipher import AES
from urllib.parse import quote

# ==== 你的參數 (請替換成表格上的資料) ====
MERCHANT_ID = "106952201270001"   # RG商代
TERMINAL_ID = "70002146"          # RG端代
STORE_CODE  = "e4f76eb8-d236-45fb-984b-92b2d1912492"  # 店家識別碼
KEY = bytes.fromhex("3abae022acd6fc873821411c0b402c0fbe90d90bdda295ed15296f8ae465bf8b")  # 端末設備Key (hex)
IV  = bytes.fromhex("12fe9f5e0c3cc7c664894f19ba265050")  # 端末設備IV (hex，記得補齊完整)
LAYMON = "https://iqrc.epay365.com.tw/"
# ==== 測試交易資料 ====
payload = {
    "set_price": "100",
    "pos_order_number": "TEST12345",   # 自訂訂單編號，不可重複
    "callback_url": "https://yourdomain.com/callback",
    "return_url": "https://yourdomain.com/return"
}

# --- AES256-CBC 加密 ---
def pad(s):
    pad_len = 16 - (len(s) % 16)
    return s + chr(pad_len) * pad_len

def unpad(s):
    return s[:-ord(s[-1])]

def encrypt_aes(data: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    ct_bytes = cipher.encrypt(pad(data).encode("utf-8"))
    return b64encode(ct_bytes).decode("utf-8")

def decrypt_aes(enc: str, key: bytes, iv: bytes) -> str:
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = cipher.decrypt(b64decode(enc))
    return unpad(pt.decode("utf-8"))

# --- 加密交易資料 ---
json_str = json.dumps(payload, separators=(',', ':'))
TransactionData = encrypt_aes(json_str, KEY, IV)
HashDigest = hashlib.sha256(TransactionData.encode("utf-8")).hexdigest()

print("加密後 TransactionData:", TransactionData)
print("校驗碼 HashDigest:", HashDigest)

# --- 發送 API (線上付款) ---
url = f"https://[雷門API網址]/calc/pay_encrypt/{STORE_CODE}"
params = {
    "TransactionData": quote(TransactionData),  # URL encode
    "HashDigest": HashDigest
}

print("請求網址：", url, params)

# ⚠️ 測試時可先列印，不要真的發送
# response = requests.get(url, params=params)
# print(response.text)


加密後 TransactionData: LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx+p3XWEk2T0jXyFz/um3DAOK9miKcbQpeA==
校驗碼 HashDigest: 98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556
請求網址： https://[雷門API網址]/calc/pay_encrypt/e4f76eb8-d236-45fb-984b-92b2d1912492 {'TransactionData': 'LwIo/f0GsJQ1mLLlWKTqJsoi5qsxSApZKfjpXToaReHNFDMcw560TLVvIOVcIt6NDQyc8YwnCu8tECEJaTJqG1FP1rI/0wYlJeQR1p1Dw8CqA84ySf6IrrHROz3beuJ3QiZF7sYbCxglkz/jbpr9RxfUrt9RQQ8zf3ZmAcWLeMs9J6ryCBIfx%2Bp3XWEk2T0jXyFz/um3DAOK9miKcbQpeA%3D%3D', 'HashDigest': '98c15532d725f6135887ce75b867333697f89406bd253fe25192768854434556'}


In [ ]:
# -*- coding: utf-8 -*-
"""
每天 08:00 自動：
1) 查詢 reservation：今天、review_status='approved'
2) 連 users 表拿 email
3) 依使用者彙整並寄出確認信

依賴：
- 你的 MySQL_Run 函式 (from MySQL import MySQL_Run)
- Gmail 應用程式密碼 (建議放在 .env)
"""

import os
from datetime import datetime
from zoneinfo import ZoneInfo
import pandas as pd
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger
from dotenv import load_dotenv
from Backend.MySQL import MySQL_Doing 
# === 載入環境變數 ===
load_dotenv()

SENDER_EMAIL = os.getenv("Sender_email")
SENDER_PASS  = os.getenv("Password_email")
TZ = ZoneInfo("Asia/Taipei")
MySQL_Doing = MySQL_Doing()

# ========== 信件樣板 ==========
MAIL_SUBJECT = "【乘車提醒】您今日的預約資訊"
MAIL_TEXT_TEMPLATE = """親愛的乘客您好，

以下為您今日 ({today}) 的預約資訊：
{lines}

若資訊有誤或需更改，請盡速與我們聯繫。祝您旅途順利！

— 花蓮小巴預約系統
"""
# ========== 郵件發送 ==========
def send_email(receiver_email: str, subject: str, text: str):
    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = SENDER_EMAIL
    message["To"] = receiver_email

    part1 = MIMEText(text, "plain", "utf-8")
    message.attach(part1)

    smtp_server = "smtp.gmail.com"
    port = 465  # SSL

    with smtplib.SMTP_SSL(smtp_server, port) as server:
        server.login(SENDER_EMAIL, SENDER_PASS)
        server.sendmail(SENDER_EMAIL, receiver_email, message.as_string())

# ========== 查詢今天預約 & 準備寄信名單 ==========
def fetch_today_reservations() -> pd.DataFrame:
    """
    回傳欄位：
      email, user_id, reservation_id, booking_time, booking_number,
      booking_start_station_name, booking_end_station_name, review_status, payment_status
    僅限今天 + approved
    """
    sql = """
    SELECT 
        u.email,
        r.user_id,
        r.reservation_id,
        r.booking_time,
        r.booking_number,
        r.booking_start_station_name,
        r.booking_end_station_name,
        r.review_status,
        r.payment_status
    FROM reservation r
    JOIN users u ON u.user_id = r.user_id
    WHERE r.review_status = 'approved'
      AND DATE(r.booking_time) = CURDATE()
      AND u.email IS NOT NULL
      AND u.email <> '';
    """
    rows = MySQL_Doing.run(sql)
    df = pd.DataFrame(rows)
    # 安全處理：若表空或欄位大小寫/命名不同，可在此做 rename
    return df

# ========== 組信內容（依 email 彙整多筆預約） ==========
def build_and_send_emails():
    now = datetime.now(TZ)
    print(f"[{now:%Y-%m-%d %H:%M:%S}] 開始檢查今日預約…")

    try:
        df = fetch_today_reservations()
    except Exception as e:
        print(f"[{datetime.now(TZ):%Y-%m-%d %H:%M:%S}] 查詢 DB 失敗：{e}")
        return

    if df.empty:
        print(f"[{datetime.now(TZ):%Y-%m-%d %H:%M:%S}] 今日無符合條件的預約。")
        return

    required_cols = {
        "email", "reservation_id", "booking_time", "booking_number",
        "booking_start_station_name", "booking_end_station_name"
    }
    missing = required_cols - set(df.columns)
    if missing:
        print(f"[{datetime.now(TZ):%Y-%m-%d %H:%M:%S}] 结果少欄位: {missing}")
        return

    grouped = df.groupby("email", dropna=True)

    success, fail = 0, 0
    for email, g in grouped:
        lines = []
        for _, r in g.iterrows():
            bt = str(r["booking_time"])
            line = (
                f"- 預約編號: {r['reservation_id']}｜"
                f"時間: {bt}｜"
                f"人數: {r['booking_number']}｜"
                f"{r['booking_start_station_name']} → {r['booking_end_station_name']}"
            )
            lines.append(line)

        body = MAIL_TEXT_TEMPLATE.format(
            today=f"{now:%Y-%m-%d}",
            lines="\n".join(lines)
        )

        try:
            send_email(email, MAIL_SUBJECT, body)
            print(f"✔ 已寄出：{email}（{len(g)} 筆）")
            success += 1
        except Exception as e:
            print(f"✘ 寄送失敗：{email} → {e}")
            fail += 1

    print(f"[{datetime.now(TZ):%Y-%m-%d %H:%M:%S}] 寄送完成：成功 {success} 位、失敗 {fail} 位。")

# ========== 主程式：每天 08:00 自動執行 ==========
def main(hour=8, minute=0):
    scheduler = BlockingScheduler(timezone=TZ)
    scheduler.add_job(
        build_and_send_emails,
        CronTrigger(hour=hour, minute=minute),
        id="daily_0800_send"
    )

    print(f"排程已啟動（Asia/Taipei）。每天 {hour:02d}:{minute:02d} 寄送今日乘車提醒。")
    print("按 Ctrl+C 可停止。")

    try:
        scheduler.start()
    except (KeyboardInterrupt, SystemExit):
        print("排程停止。")

if __name__ == "__main__":
    main(hour=8, minute=00)

# 321